In [1]:
import scipy.io
import torch
import numpy as np
import time
import os

In [2]:
result = scipy.io.loadmat('./duke/pytorch_result.mat')

In [3]:
def evaluate(qf,ql,qc,gf,gl,gc):
    query = qf
    score = np.dot(gf,query)
    
    index = np.argsort(score)  #from small to large
    index = index[::-1]
    
    query_index = np.argwhere(gl==ql)
    camera_index = np.argwhere(gc==qc)

    good_index = np.setdiff1d(query_index, camera_index, assume_unique=True)
    junk_index1 = np.argwhere(gl==-1)
    junk_index2 = np.intersect1d(query_index, camera_index)
    junk_index = np.append(junk_index2, junk_index1) #.flatten())
    
    CMC_tmp = compute_mAP(index, good_index, junk_index)
    return CMC_tmp

In [4]:
def compute_mAP(index, good_index, junk_index):
    ap = 0
    cmc = torch.IntTensor(len(index)).zero_()
    if good_index.size==0:   # if empty
        cmc[0] = -1
        return ap,cmc

    # remove junk_index
    mask = np.in1d(index, junk_index, invert=True)
    index = index[mask]

    # find good_index index
    ngood = len(good_index)
    mask = np.in1d(index, good_index)
    rows_good = np.argwhere(mask==True)
    rows_good = rows_good.flatten()
    
    cmc[rows_good[0]:] = 1
    for i in range(ngood):
        d_recall = 1.0/ngood
        precision = (i+1)*1.0/(rows_good[i]+1)
        if rows_good[i]!=0:
            old_precision = i*1.0/rows_good[i]
        else:
            old_precision=1.0
        ap = ap + d_recall*(old_precision + precision)/2

    return ap, cmc

In [5]:
query_feature = result['query_f']
query_cam = result['query_cam'][0]
query_label = result['query_label'][0]
gallery_feature = result['gallery_f']
gallery_cam = result['gallery_cam'][0]
gallery_label = result['gallery_label'][0]

In [6]:
CMC = torch.IntTensor(len(gallery_label)).zero_()
ap = 0.0
for i in range(len(query_label)):
    ap_tmp, CMC_tmp = evaluate(query_feature[i],query_label[i],query_cam[i],gallery_feature,gallery_label,gallery_cam)
    if CMC_tmp[0]==-1:
        continue
    CMC = CMC + CMC_tmp
    ap += ap_tmp
    print(i, CMC_tmp[0])

CMC = CMC.float()
CMC = CMC/len(query_label) #average CMC
print('Rank@1:%f Rank@5:%f Rank@10:%f mAP:%f'%(CMC[0],CMC[4],CMC[9],ap/len(query_label)))

0 tensor(1, dtype=torch.int32)
1 tensor(1, dtype=torch.int32)
2 tensor(0, dtype=torch.int32)
3 tensor(0, dtype=torch.int32)
4 tensor(1, dtype=torch.int32)
5 tensor(1, dtype=torch.int32)
6 tensor(1, dtype=torch.int32)
7 tensor(1, dtype=torch.int32)
8 tensor(1, dtype=torch.int32)
9 tensor(1, dtype=torch.int32)
10 tensor(1, dtype=torch.int32)
11 tensor(1, dtype=torch.int32)
12 tensor(1, dtype=torch.int32)
13 tensor(1, dtype=torch.int32)
14 tensor(1, dtype=torch.int32)
15 tensor(1, dtype=torch.int32)
16 tensor(1, dtype=torch.int32)
17 tensor(0, dtype=torch.int32)
18 tensor(1, dtype=torch.int32)
19 tensor(1, dtype=torch.int32)
20 tensor(1, dtype=torch.int32)
21 tensor(1, dtype=torch.int32)
22 tensor(1, dtype=torch.int32)
23 tensor(1, dtype=torch.int32)
24 tensor(0, dtype=torch.int32)
25 tensor(1, dtype=torch.int32)
26 tensor(1, dtype=torch.int32)
27 tensor(1, dtype=torch.int32)
28 tensor(1, dtype=torch.int32)
29 tensor(1, dtype=torch.int32)
30 tensor(1, dtype=torch.int32)
31 tensor(1, dtype

291 tensor(1, dtype=torch.int32)
292 tensor(1, dtype=torch.int32)
293 tensor(0, dtype=torch.int32)
294 tensor(1, dtype=torch.int32)
295 tensor(0, dtype=torch.int32)
296 tensor(1, dtype=torch.int32)
297 tensor(1, dtype=torch.int32)
298 tensor(1, dtype=torch.int32)
299 tensor(1, dtype=torch.int32)
300 tensor(1, dtype=torch.int32)
301 tensor(0, dtype=torch.int32)
302 tensor(0, dtype=torch.int32)
303 tensor(0, dtype=torch.int32)
304 tensor(1, dtype=torch.int32)
305 tensor(1, dtype=torch.int32)
306 tensor(0, dtype=torch.int32)
307 tensor(1, dtype=torch.int32)
308 tensor(0, dtype=torch.int32)
309 tensor(1, dtype=torch.int32)
310 tensor(1, dtype=torch.int32)
311 tensor(1, dtype=torch.int32)
312 tensor(1, dtype=torch.int32)
313 tensor(0, dtype=torch.int32)
314 tensor(1, dtype=torch.int32)
315 tensor(1, dtype=torch.int32)
316 tensor(1, dtype=torch.int32)
317 tensor(1, dtype=torch.int32)
318 tensor(1, dtype=torch.int32)
319 tensor(1, dtype=torch.int32)
320 tensor(1, dtype=torch.int32)
321 tensor

549 tensor(0, dtype=torch.int32)
550 tensor(1, dtype=torch.int32)
551 tensor(1, dtype=torch.int32)
552 tensor(0, dtype=torch.int32)
553 tensor(1, dtype=torch.int32)
554 tensor(1, dtype=torch.int32)
555 tensor(1, dtype=torch.int32)
556 tensor(1, dtype=torch.int32)
557 tensor(1, dtype=torch.int32)
558 tensor(1, dtype=torch.int32)
559 tensor(1, dtype=torch.int32)
560 tensor(0, dtype=torch.int32)
561 tensor(1, dtype=torch.int32)
562 tensor(1, dtype=torch.int32)
563 tensor(1, dtype=torch.int32)
564 tensor(0, dtype=torch.int32)
565 tensor(1, dtype=torch.int32)
566 tensor(1, dtype=torch.int32)
567 tensor(1, dtype=torch.int32)
568 tensor(0, dtype=torch.int32)
569 tensor(1, dtype=torch.int32)
570 tensor(1, dtype=torch.int32)
571 tensor(1, dtype=torch.int32)
572 tensor(1, dtype=torch.int32)
573 tensor(0, dtype=torch.int32)
574 tensor(0, dtype=torch.int32)
575 tensor(0, dtype=torch.int32)
576 tensor(0, dtype=torch.int32)
577 tensor(0, dtype=torch.int32)
578 tensor(0, dtype=torch.int32)
579 tensor

811 tensor(1, dtype=torch.int32)
812 tensor(1, dtype=torch.int32)
813 tensor(1, dtype=torch.int32)
814 tensor(0, dtype=torch.int32)
815 tensor(1, dtype=torch.int32)
816 tensor(1, dtype=torch.int32)
817 tensor(1, dtype=torch.int32)
818 tensor(1, dtype=torch.int32)
819 tensor(0, dtype=torch.int32)
820 tensor(0, dtype=torch.int32)
821 tensor(1, dtype=torch.int32)
822 tensor(1, dtype=torch.int32)
823 tensor(1, dtype=torch.int32)
824 tensor(1, dtype=torch.int32)
825 tensor(1, dtype=torch.int32)
826 tensor(1, dtype=torch.int32)
827 tensor(1, dtype=torch.int32)
828 tensor(1, dtype=torch.int32)
829 tensor(1, dtype=torch.int32)
830 tensor(1, dtype=torch.int32)
831 tensor(1, dtype=torch.int32)
832 tensor(1, dtype=torch.int32)
833 tensor(1, dtype=torch.int32)
834 tensor(1, dtype=torch.int32)
835 tensor(1, dtype=torch.int32)
836 tensor(1, dtype=torch.int32)
837 tensor(1, dtype=torch.int32)
838 tensor(1, dtype=torch.int32)
839 tensor(0, dtype=torch.int32)
840 tensor(1, dtype=torch.int32)
841 tensor

1058 tensor(1, dtype=torch.int32)
1059 tensor(1, dtype=torch.int32)
1060 tensor(1, dtype=torch.int32)
1061 tensor(1, dtype=torch.int32)
1062 tensor(1, dtype=torch.int32)
1063 tensor(1, dtype=torch.int32)
1064 tensor(1, dtype=torch.int32)
1065 tensor(1, dtype=torch.int32)
1066 tensor(1, dtype=torch.int32)
1067 tensor(0, dtype=torch.int32)
1068 tensor(0, dtype=torch.int32)
1069 tensor(0, dtype=torch.int32)
1070 tensor(1, dtype=torch.int32)
1071 tensor(0, dtype=torch.int32)
1072 tensor(0, dtype=torch.int32)
1073 tensor(0, dtype=torch.int32)
1074 tensor(0, dtype=torch.int32)
1075 tensor(1, dtype=torch.int32)
1076 tensor(0, dtype=torch.int32)
1077 tensor(1, dtype=torch.int32)
1078 tensor(1, dtype=torch.int32)
1079 tensor(1, dtype=torch.int32)
1080 tensor(1, dtype=torch.int32)
1081 tensor(1, dtype=torch.int32)
1082 tensor(1, dtype=torch.int32)
1083 tensor(1, dtype=torch.int32)
1084 tensor(0, dtype=torch.int32)
1085 tensor(1, dtype=torch.int32)
1086 tensor(1, dtype=torch.int32)
1087 tensor(1,

1329 tensor(1, dtype=torch.int32)
1330 tensor(1, dtype=torch.int32)
1331 tensor(1, dtype=torch.int32)
1332 tensor(1, dtype=torch.int32)
1333 tensor(1, dtype=torch.int32)
1334 tensor(1, dtype=torch.int32)
1335 tensor(0, dtype=torch.int32)
1336 tensor(0, dtype=torch.int32)
1337 tensor(1, dtype=torch.int32)
1338 tensor(0, dtype=torch.int32)
1339 tensor(1, dtype=torch.int32)
1340 tensor(1, dtype=torch.int32)
1341 tensor(1, dtype=torch.int32)
1342 tensor(1, dtype=torch.int32)
1343 tensor(1, dtype=torch.int32)
1344 tensor(1, dtype=torch.int32)
1345 tensor(1, dtype=torch.int32)
1346 tensor(1, dtype=torch.int32)
1347 tensor(0, dtype=torch.int32)
1348 tensor(1, dtype=torch.int32)
1349 tensor(1, dtype=torch.int32)
1350 tensor(1, dtype=torch.int32)
1351 tensor(1, dtype=torch.int32)
1352 tensor(1, dtype=torch.int32)
1353 tensor(1, dtype=torch.int32)
1354 tensor(1, dtype=torch.int32)
1355 tensor(0, dtype=torch.int32)
1356 tensor(1, dtype=torch.int32)
1357 tensor(1, dtype=torch.int32)
1358 tensor(0,

1605 tensor(1, dtype=torch.int32)
1606 tensor(1, dtype=torch.int32)
1607 tensor(1, dtype=torch.int32)
1608 tensor(1, dtype=torch.int32)
1609 tensor(0, dtype=torch.int32)
1610 tensor(0, dtype=torch.int32)
1611 tensor(1, dtype=torch.int32)
1612 tensor(1, dtype=torch.int32)
1613 tensor(0, dtype=torch.int32)
1614 tensor(0, dtype=torch.int32)
1615 tensor(1, dtype=torch.int32)
1616 tensor(1, dtype=torch.int32)
1617 tensor(1, dtype=torch.int32)
1618 tensor(1, dtype=torch.int32)
1619 tensor(0, dtype=torch.int32)
1620 tensor(1, dtype=torch.int32)
1621 tensor(1, dtype=torch.int32)
1622 tensor(1, dtype=torch.int32)
1623 tensor(1, dtype=torch.int32)
1624 tensor(1, dtype=torch.int32)
1625 tensor(1, dtype=torch.int32)
1626 tensor(1, dtype=torch.int32)
1627 tensor(1, dtype=torch.int32)
1628 tensor(0, dtype=torch.int32)
1629 tensor(1, dtype=torch.int32)
1630 tensor(1, dtype=torch.int32)
1631 tensor(1, dtype=torch.int32)
1632 tensor(1, dtype=torch.int32)
1633 tensor(0, dtype=torch.int32)
1634 tensor(0,

1889 tensor(0, dtype=torch.int32)
1890 tensor(1, dtype=torch.int32)
1891 tensor(0, dtype=torch.int32)
1892 tensor(1, dtype=torch.int32)
1893 tensor(1, dtype=torch.int32)
1894 tensor(0, dtype=torch.int32)
1895 tensor(0, dtype=torch.int32)
1896 tensor(1, dtype=torch.int32)
1897 tensor(0, dtype=torch.int32)
1898 tensor(1, dtype=torch.int32)
1899 tensor(1, dtype=torch.int32)
1900 tensor(1, dtype=torch.int32)
1901 tensor(1, dtype=torch.int32)
1902 tensor(1, dtype=torch.int32)
1903 tensor(1, dtype=torch.int32)
1904 tensor(1, dtype=torch.int32)
1905 tensor(0, dtype=torch.int32)
1906 tensor(0, dtype=torch.int32)
1907 tensor(1, dtype=torch.int32)
1908 tensor(1, dtype=torch.int32)
1909 tensor(1, dtype=torch.int32)
1910 tensor(1, dtype=torch.int32)
1911 tensor(1, dtype=torch.int32)
1912 tensor(1, dtype=torch.int32)
1913 tensor(1, dtype=torch.int32)
1914 tensor(1, dtype=torch.int32)
1915 tensor(1, dtype=torch.int32)
1916 tensor(1, dtype=torch.int32)
1917 tensor(1, dtype=torch.int32)
1918 tensor(0,

2155 tensor(1, dtype=torch.int32)
2156 tensor(0, dtype=torch.int32)
2157 tensor(0, dtype=torch.int32)
2158 tensor(0, dtype=torch.int32)
2159 tensor(0, dtype=torch.int32)
2160 tensor(1, dtype=torch.int32)
2161 tensor(0, dtype=torch.int32)
2162 tensor(0, dtype=torch.int32)
2163 tensor(1, dtype=torch.int32)
2164 tensor(0, dtype=torch.int32)
2165 tensor(1, dtype=torch.int32)
2166 tensor(1, dtype=torch.int32)
2167 tensor(1, dtype=torch.int32)
2168 tensor(1, dtype=torch.int32)
2169 tensor(1, dtype=torch.int32)
2170 tensor(1, dtype=torch.int32)
2171 tensor(0, dtype=torch.int32)
2172 tensor(1, dtype=torch.int32)
2173 tensor(1, dtype=torch.int32)
2174 tensor(1, dtype=torch.int32)
2175 tensor(0, dtype=torch.int32)
2176 tensor(1, dtype=torch.int32)
2177 tensor(1, dtype=torch.int32)
2178 tensor(1, dtype=torch.int32)
2179 tensor(1, dtype=torch.int32)
2180 tensor(1, dtype=torch.int32)
2181 tensor(0, dtype=torch.int32)
2182 tensor(1, dtype=torch.int32)
2183 tensor(0, dtype=torch.int32)
2184 tensor(1,